In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import time
import datetime
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.svm import SVC
from sklearn import preprocessing


In [ ]:
### Модуль с классификацией ###

# Загрузка составленного датасета из файла

data_class = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Diplom Work/data/DF_res_class.csv', delimiter=',')
#data_clast

data_class = data_class.drop(columns = ['Unnamed: 0'], axis = 1)
data_class

,equal,bracket_r,comment,grid,quat_mark,word_from,point,comma,word_select,word_union,star,word_where,word_or,line,percent,point_comma,dollar,plus,at,Label
0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,3,5,0,0,3,2,2,1,2,0,0,1,0,0,0,0,0,0,0,1
2,2,1,1,0,3,1,0,1,2,1,1,1,1,0,0,0,0,0,2,1
3,1,1,1,1,0,1,0,1,2,1,1,1,1,0,0,0,0,0,0,1
4,2,1,1,0,1,2,0,0,2,0,0,2,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30604,1,0,0,0,2,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0
30605,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
30606,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
30607,0,1,0,0,0,2,0,0,2,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
X = data_class.drop(['Label'], axis = 1)
normalized_df=(X-X.mean())/X.std()
X = normalized_df
y = data_class['Label']
# разбиваем на тестовую и обучающую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
#находим лучшие параметры для kNN
params = {'n_neighbors' : [3,4,5,6,7,8],
         'weights' : ['uniform', 'distance'],
         'p' : [2,3,4]}
scoring = {'f1', 'accuracy'}

knn = KNeighborsClassifier() 
gs = GridSearchCV(knn, params, scoring = scoring, refit='f1', cv = 3, return_train_score=True)
gs.fit(X_train, y_train)
print(gs.best_params_) #лучшие параметры для этой модели получились: {'n_neighbors': 4, 'p': 3, 'weights': 'distance'}

{'n_neighbors': 4, 'p': 3, 'weights': 'distance'}


In [ ]:
#находим лучшие параметры для BDTC
params = {'n_estimators': [10, 25, 50, 75, 100, 125, 150],
         'max_samples':[0.05, 0.1, 0.3, 0.5, 0.7, 0.9]
        }
scoring = {'f1', 'accuracy'}
dtc = DecisionTreeClassifier(random_state = 42)
bagging = BaggingClassifier(estimator = dtc, random_state = 41)
gs = GridSearchCV(bagging, params, scoring = scoring, refit='f1', cv = 3, return_train_score=True)
gs.fit(X_train, y_train)
print(gs.best_params_) #лучшие параметры для этой модели получились: {'max_samples': 0.9, 'n_estimators': 150}


{'max_samples': 0.9, 'n_estimators': 150}


In [ ]:
#находим лучшие параметры для AdaBoost
params = {'n_estimators': [10, 25, 50, 75, 100, 125, 150],
        'learning_rate':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.2]
        }
scoring = {'f1', 'accuracy'}
boosting = AdaBoostClassifier(estimator = dtc, random_state = 41)
gs = GridSearchCV(boosting, params, scoring = scoring, refit='f1', cv = 3, return_train_score=True)
gs.fit(X_train, y_train)
print(gs.best_params_) #лучшие параметры для этой модели получились: {'learning_rate': 0.9, 'n_estimators': 10}

In [ ]:
#находим лучшие параметры для RFC
params = {'n_estimators': [10, 30, 50, 70, 90],
             'min_samples_leaf': [3, 5, 7, 9],
             'max_features': [3, 5, 7, 9, 11], 
             'max_depth': [ 10, 15, 20, 25]}
scoring = {'f1', 'accuracy'}
rfc = RandomForestClassifier(random_state = 42)
gs = GridSearchCV(rfc, params, scoring = scoring, refit='f1', cv = 3, return_train_score=True)
gs.fit(X_train, y_train)
print(gs.best_params_) #лучшие параметры для этой модели получились: {'max_depth': 25, 'max_features': 3, 'min_samples_leaf': 3, 'n_estimators': 90}

In [ ]:
#находим лучшие параметры для Логистической регрессии
params = {'tol': [0.001, 0.0001, 0.00001, 0.000001],
         'C': [0.3, 0.5, 0.7, 1.0, 1.3, 1.5, 1.7, 2.0],
         'intercept_scaling': [0.7, 1.0, 1.3, 1.5, 1.7, 2.0], 
         'max_iter': [150, 200, 250, 300]}
scoring = {'f1', 'accuracy'}
lr = LogisticRegression(random_state = 42)
gs = GridSearchCV(lr, params, scoring = scoring, refit='f1', cv = 3, return_train_score=True)
gs.fit(X_train, y_train)
print(gs.best_params_) #лучшие параметры для этой модели получились: {'C': 2.0, 'intercept_scaling': 0.7, 'max_iter': 150, 'tol': 0.001}

{'C': 2.0, 'intercept_scaling': 0.7, 'max_iter': 150, 'tol': 0.001}


In [ ]:
from sklearn import svm
#находим лучшие параметры для метода опорных векторов
params = {'gamma': ['scale', 'auto'],
         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
         'degree': [2, 3, 4, 5, 6, 7]}
scoring = {'f1', 'accuracy'}
M_svm = svm.NuSVC(random_state = 42)
gs = GridSearchCV(M_svm, params, scoring = scoring, refit='f1', cv = 3, return_train_score=True)
gs.fit(X_train, y_train)
print(gs.best_params_) #лучшие параметры для этой модели получились: {'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}

{'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
#Выбор моделей (6 штук) - 1)kNN 2)Bagg(DTC) 3) Boost(DTC) 4) RFC 5) LR 6)SVM

start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'n_neighbors': 4, 'p': 3, 'weights': 'distance'}
knn = KNeighborsClassifier(n_neighbors = 4, weights = 'distance', p = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(f"---->Time of kNN, k = 4 : {datetime.datetime.now() - start_time}")
print('accuracy of kNN: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of kNN: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of kNN: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of kNN: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'max_samples': 0.9, 'n_estimators': 150}
bagging = BaggingClassifier(estimator = dtc, random_state = 41, n_estimators = 150, max_samples = 0.9)
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
print(f"---->Time of Bag : {datetime.datetime.now() - start_time}")
print('accuracy of Bag: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of Bag: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of Bag: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of Bag: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'learning_rate': 0.9, 'n_estimators': 10}
boosting = AdaBoostClassifier(estimator = dtc, random_state = 41, n_estimators = 10, learning_rate = 0.9)
boosting.fit(X_train, y_train)
y_pred = boosting.predict(X_test)
print(f"---->Time of Boost : {datetime.datetime.now() - start_time}")
print('accuracy of Boost: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of Boost: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of Boost: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of Boost: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'max_depth': 25, 'max_features': 3, 'min_samples_leaf': 3, 'n_estimators': 90}
rfc = RandomForestClassifier(n_estimators = 90, min_samples_leaf = 3, max_features = 3, max_depth = 25, random_state = 42)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(f"---->Time of RFC : {datetime.datetime.now() - start_time}")
print('accuracy of RFC: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of RFC: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of RFC: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of RFC: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'C': 2.0, 'intercept_scaling': 0.7, 'max_iter': 150, 'tol': 0.001}
lr = LogisticRegression(tol = 0.001, C = 2.0, intercept_scaling = 0.7, max_iter = 150, random_state = 42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(f"---->Time of LR : {datetime.datetime.now() - start_time}")
print('accuracy of LR: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of LR: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of LR: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of LR: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
M_svm = svm.SVC(degree = 2, gamma = 'scale', kernel = 'rbf', random_state = 42)
M_svm.fit(X_train, y_train)
y_pred = M_svm.predict(X_test)
print(f"---->Time of LR : {datetime.datetime.now() - start_time}")
print('accuracy of LR: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of LR: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of LR: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of LR: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

---->Time of kNN, k = 4 : 0:01:05.403314
accuracy of kNN:  0.9952085375149733
precision of kNN:  0.9985233313644418
recall of kNN:  0.9885964912280701
f1 measure of kNN:  0.9935351160740523
---->Time of Bag : 0:00:03.303750
accuracy of Bag:  0.9970597843841882
precision of Bag:  0.9985307081986482
recall of Bag:  0.9935672514619883
f1 measure of Bag:  0.9960427964238604
---->Time of Boost : 0:00:00.339288
accuracy of Boost:  0.9968419906348688
precision of Boost:  0.9982368498383779
recall of Boost:  0.9932748538011696
f1 measure of Boost:  0.9957496702330354
---->Time of RFC : 0:00:00.836940
accuracy of RFC:  0.9971686812588478
precision of RFC:  0.9982384028185555
recall of RFC:  0.9941520467836257
f1 measure of RFC:  0.9961910342806914
---->Time of LR : 0:00:00.170125
accuracy of LR:  0.9630839594903626
precision of LR:  0.9764924218991649
recall of LR:  0.9230994152046783
f1 measure of LR:  0.9490455433638959


In [ ]:
start_time = datetime.datetime.now()
#Подставляем лучшие параметры {'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
M_svm = svm.SVC(degree = 2, gamma = 'scale', kernel = 'rbf', random_state = 42)
M_svm.fit(X_train, y_train)
y_pred = M_svm.predict(X_test)
print(f"---->Time of LR : {datetime.datetime.now() - start_time}")
print('accuracy of LR: ', accuracy_score(y_test, y_pred)) #Доля верных ответов, Accuracy = TP + TN / (TP + TN + FP + FN)
print('precision of LR: ', precision_score(y_test, y_pred)) #точность, то есть на сколько можно доверять классификатору, Precision = TP / (TP + FP)
print('recall of LR: ', recall_score(y_test, y_pred)) #как много объектов класса "аномальный трафик" определяет классификатор, Recall = TP / (TP + FN)
print('f1 measure of LR: ', f1_score(y_test, y_pred)) #гармоническое среднее между точностью и полнотой, F1 = 2 * (precision * recall) / (precision + recall)

---->Time of LR : 0:00:01.797638
accuracy of LR:  0.9947729500163345
precision of LR:  0.9991119005328597
recall of LR:  0.9868421052631579
f1 measure of LR:  0.9929390997352163
